# OpenFold Colab

Runs a simplified version of [OpenFold](https://github.com/aqlaboratory/openfold) on a target sequence. Adapted from DeepMind's [official AlphaFold Colab](https://colab.research.google.com/github/deepmind/alphafold/blob/main/notebooks/AlphaFold.ipynb).

**Differences to AlphaFold v2.0**

OpenFold is a trainable PyTorch reimplementation of AlphaFold 2. For the purposes of inference, it is practically identical to the original ("practically" because ensembling is excluded from OpenFold (recycling is enabled, however)).

In this notebook, OpenFold is run with your choice of our original OpenFold parameters or DeepMind's publicly released parameters for AlphaFold 2.

**Note**

Like DeepMind's official Colab, this notebook uses **no templates (homologous structures)** and a selected portion of the full [BFD database](https://bfd.mmseqs.com/).

**Citing this work**

Any publication that discloses findings arising from using this notebook should [cite](https://github.com/deepmind/alphafold/#citing-this-work) DeepMind's [AlphaFold paper](https://doi.org/10.1038/s41586-021-03819-2).

**Licenses**

This Colab supports inference with the [AlphaFold model parameters](https://github.com/deepmind/alphafold/#model-parameters-license), made available under the Creative Commons Attribution 4.0 International ([CC BY 4.0](https://creativecommons.org/licenses/by/4.0/legalcode)) license. The Colab itself is provided under the [Apache 2.0 license](https://www.apache.org/licenses/LICENSE-2.0). See the full license statement below.

**More information**

You can find more information about how AlphaFold/OpenFold works in DeepMind's two Nature papers:

*   [AlphaFold methods paper](https://www.nature.com/articles/s41586-021-03819-2)
*   [AlphaFold predictions of the human proteome paper](https://www.nature.com/articles/s41586-021-03828-1)

FAQ on how to interpret AlphaFold/OpenFold predictions are [here](https://alphafold.ebi.ac.uk/faq).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import py3Dmol
from proteome import protein
from proteome.models.folding.openfold.modeling import OpenFoldForFolding
from proteome.models.folding.openfold.np.relax import relax

/home/conradry71/micromamba/envs/proteome/lib/python3.10/site-packages/Bio/Data/SCOPData.py:18: BiopythonDeprecationWarning: The 'Bio.Data.SCOPData' module will be deprecated in a future release of Biopython in favor of 'Bio.Data.PDBData.
  warnings.warn(
/home/conradry71/micromamba/envs/proteome/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sequence = 'MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH'

In [4]:
folder = OpenFoldForFolding("finetuning-3")
predicted_protein, plddt = folder.fold(sequence)

Running jackhmmer on uniref90 database...
Running jackhmmer on smallbfd database...
Running jackhmmer on mgnify database...
1 Sequences Found in uniref90
39 Sequences Found in smallbfd
1 Sequences Found in mgnify


In [5]:
unrelaxed_protein_dict = {k: getattr(predicted_protein, k).cpu().numpy() for k in ['atom_positions', 'aatype', 'atom_mask', 'residue_index', 'b_factors']}
unrelaxed_protein_dict['aatype'] = unrelaxed_protein_dict['aatype'][:, 0]
unrelaxed_protein_dict['residue_index'] = unrelaxed_protein_dict['residue_index'][:, 0]
unrelaxed_protein = protein.Protein(**unrelaxed_protein_dict)

In [6]:
unrelaxed_pdb = protein.to_pdb(unrelaxed_protein)

In [7]:
amber_relaxer = relax.AmberRelaxation(
    max_iterations=0,
    tolerance=2.39,
    stiffness=10.0,
    exclude_residues=[],
    max_outer_iterations=20,
    use_gpu=False,
)
relaxed_pdb, _, _ = amber_relaxer.process(
    prot=unrelaxed_protein, cif_output=False
)

In [8]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(relaxed_pdb)
color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol